### Framework imports

In [ ]:
from noronha.tools.serving import LazyModelServer
from noronha.tools.shortcuts import model_path

### Application imports

In [ ]:
import json
import os
import numpy as np
from sklearn.externals import joblib

### Loading the model

In [ ]:
def load(path):
    clf_path = os.path.join(path, 'clf.pkl')
    clf = joblib.load(clf_path)
    return clf

### Defining the prediction function

In [ ]:
def predict(x, clf):
    features = json.loads(x)
    features = np.array(features).reshape(1, -1)
    return clf.predict(features)[0]

In [ ]:
import json
import warnings
from abc import ABC, abstractmethod
from datetime import datetime
from flask import Flask, request
from werkzeug.serving import run_simple

from noronha.common.constants import DateFmt, OnlineConst, Task
from noronha.common.errors import NhaDataError, PrettyError, MisusageError
from noronha.common.logging import LOG
from noronha.common.parser import assert_json, assert_str, StructCleaner
from noronha.db.depl import Deployment
from noronha.tools.shortcuts import require_movers
from noronha.tools.utils import load_proc_monitor, HistoryQueue
from noronha.tools.serving import ModelServer


In [ ]:
class LazyModelServer(ModelServer):  # TODO: better logs and error messages that the end user can receive
    
    def __init__(self, predict_func, load_model_func, model_name: str = None, max_models: int = 100):
        
        assert callable(load_model_func)
        super().__init__(predict_func=predict_func, enrich=False)
        self._load_model_func = load_model_func
        self._model_name = model_name  # TODO: if not provided, resolve by shortcut model_meta
        self._max_models = max_models
        self._loaded_models = {}
        self._model_usage = HistoryQueue(max_size=max_models)
    
    def purge_model(self):
        
        least_used = self._model_usage.get()
        _ = self._loaded_models.pop(least_used)
    
    def load_model(self, version):
        
        if len(self._loaded_models) >= self._max_models:
            self.purge_model()
            self.load_model(version)
        else:  # TODO: before requiring, check if exists locally
            path = require_movers(model=self._model_name, version=version)
            self._loaded_models[version] = self._load_model_func(path)
    
    def fetch_model(self, version):
        
        if version not in self._loaded_models:
            self.load_model(version)
        
        self._model_usage.put(version)
        return self._loaded_models[version]
        
    def make_result(self, body, args):
        
        model = self.fetch_model(args['model_version'])
        return self._predict_func(body, model)
    
    def make_metadata(self, body, args):
        
        raise MisusageError(
            "Inference metadata for {} is ambiguous"
            .format(self.__class__.__name__)
        )
    
    def make_request_kwargs(self):
        
        body = request.get_data()
        print(body)
        charset = request.mimetype_params.get('charset') or OnlineConst.DEFAULT_CHARSET
        print(charset)
        print(dir(request))
        try:
            args = request.args
            print(args)
            print(type(args))
        except Exception as e:
            print(e)
            print(repr(e))
        return dict(
            body=body.decode(charset, 'replace'),
            args=request.args
        )

### Creating the prediction service

In [ ]:
server = LazyModelServer(
    predict,
    load,
    model_name='iris-clf',
    max_models=1
)

server()